


#**Preparazione ambiente**




In [1]:
# scaricamento dataset
!wget https://www.dropbox.com/s/c58ol588mbbdbqn/new_car_image_data.zip
# estrazione
!unzip new_car_image_data.zip

Output streaming troncato alle ultime 5000 righe.
  inflating: vehicles/4414.png       
  inflating: vehicles/4415.png       
  inflating: vehicles/4416.png       
  inflating: vehicles/4417.png       
  inflating: vehicles/4418.png       
  inflating: vehicles/4419.png       
  inflating: vehicles/442.png        
  inflating: vehicles/4420.png       
  inflating: vehicles/4421.png       
  inflating: vehicles/4422.png       
  inflating: vehicles/4423.png       
  inflating: vehicles/4424.png       
  inflating: vehicles/4425.png       
  inflating: vehicles/4426.png       
  inflating: vehicles/4427.png       
  inflating: vehicles/4428.png       
  inflating: vehicles/4429.png       
  inflating: vehicles/443.png        
  inflating: vehicles/4430.png       
  inflating: vehicles/4431.png       
  inflating: vehicles/4432.png       
  inflating: vehicles/4433.png       
  inflating: vehicles/4434.png       
  inflating: vehicles/4435.png       
  inflating: vehicles/4436.png       


In [2]:
import numpy as np
from numpy import save
import pickle
import cv2
import os
import glob
from skimage.feature import hog
from skimage import data, exposure
from IPython.display import HTML, display
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
#enable_colab()

---

#**Estrazione Hog Features**

In [3]:
# progress bar
def progress(value, max):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))



# ritorna numero elementi presenti del path specificato
def count_file(dir):
  onlyfiles = next(os.walk(dir))[2] 
  return len(onlyfiles)



# cambia spazio colore all'immagine
def change_color_space(img,n_color):
  if n_color==1:
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  elif n_color==2:
    img=cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
  elif n_color==3:
    img=cv2.cvtColor(img,cv2.COLOR_BGR2LUV)
  elif n_color==4:
    img=cv2.cvtColor(img,cv2.COLOR_BGR2HLS)
  elif n_color==5:
    img=cv2.cvtColor(img,cv2.COLOR_BGR2YUV)
  elif n_color==6:
    img=cv2.cvtColor(img,cv2.COLOR_BGR2YCrCb)
  
  return img



# estrai hog features
def extract_hog_features(img,orientations,pixels_per_cell,cells_per_block):
  hog_feat=[]
  # per ogni canale di colore dell'immagine
  for ch in range(img.shape[2]):
    # ricavo vettore delle caratteristiche dal ch-esimo canale
    fd=hog(img[:,:,ch],orientations=orientations,pixels_per_cell=pixels_per_cell,cells_per_block=cells_per_block,visualize=False,transform_sqrt=True,feature_vector=True)
    #aggiungo alla lista
    hog_feat.append(fd)
  
  return np.ravel(hog_feat)



def extract_features(path,color,orientations,pixels_per_cell,cells_per_block):
  #ricavo numero di immagini da elaborare
  file_number=count_file(path)
  # inizializzo barra progressione
  idx=0
  out = display(progress(0,file_number), display_id=True)
  # creo struttura per memorizzare features
  features=[]

  # per ogni immagine
  for im_path in glob.glob(os.path.join(path, "*")):
    #apro
    img=cv2.imread(im_path)
    # converto spazio colore, se richiesto
    if color!=None:
      img=change_color_space(img,color)
    #ricavo features vector
    fd= extract_hog_features(img,orientations,pixels_per_cell,cells_per_block)
    #aggiungo alla lista
    features.append(fd)
    idx+=1
    out.update(progress(idx, file_number))
  return np.array(features)

# salva in file .npy
def save_npy(data, name):
  print('Saving',name)
  np.save(name,data)


In [4]:
# Parametri
################################################################################
color=6 # color: 1: RGB, 2: HSV, 3: LUV, 4: HLS, 5: YUV, 6: YCrCb
orientations=9
pixels_per_cell=(8,8)
cells_per_block=(2,2)
size_test_set=0.15
################################################################################


#creazione del dataset per training
print('-> Creating features data:')
print('Extracting from positive sample')
pos_feat=extract_features('/content/vehicles',color,orientations,pixels_per_cell,cells_per_block)
print('Extracting from negative sample')
neg_feat=extract_features('/content/non-vehicles',color,orientations,pixels_per_cell,cells_per_block)

# creo train sample
data=np.vstack((pos_feat, neg_feat)).astype(np.float64) 

# normalizzo valori 0-1
scaler = MinMaxScaler()
data = scaler.fit_transform(data)
pickle.dump(scaler, open('scaler.pkl', 'wb'))

# creo etichette
label=np.hstack((np.ones(len(pos_feat)), np.zeros(len(neg_feat))))
# salvo
save_npy(data,'Data')
save_npy(label,'Label')

# divido in train e test set
state = np.random.randint(8)
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=size_test_set, random_state=state)

#salvo 
save_npy(X_train,'Train_data')
save_npy(y_train,'Train_label')
save_npy(X_test,'Test_data')
save_npy(y_test,'Test_label')


-> Creating features data:
Extracting from positive sample


Extracting from negative sample


Saving Data
Saving Label
Saving Train_data
Saving Train_label
Saving Test_data
Saving Test_label


--- ---